In [ ]:
import pandas as pd
from dcef.dcef_widget import DCEFWidget

In [ ]:
df = pd.read_csv('./examples/data/2014-01-citibike-tripdata.csv')
w = DCEFWidget(df)
w

In [ ]:
w.operation_results['transformed_df'].keys()

In [ ]:
w.operation_results['transformed_df']['schema']['fields']
tdf = w.operation_results['transformed_df']
fnames = [ f['name'] for f in tdf['schema']['fields'] ]
fnames

In [ ]:
'stoptime' in fnames 

In [ ]:
from dcef.all_transforms import Command
from dcef.lispy import s
#Here we start adding commands to the DCF Widget.  Every call to add_command replaces a command with the same name
@w.add_command
class GroupBy2(Command):
    command_default = [s("groupby2"), s('df'), 'col', {}]
    command_pattern = [[3, 'colMap', 'colEnum', ['null', 'sum', 'mean', 'median', 'count']]]
    @staticmethod 
    def transform(df, col, col_spec):
        grps = df.groupby(col)
        df_contents = {}
        for k, v in col_spec.items():
            if v == "sum":
                df_contents[k] = grps[k].apply(lambda x: x.sum())
            elif v == "mean":
                df_contents[k] = grps[k].apply(lambda x: x.mean())
            elif v == "median":
                df_contents[k] = grps[k].apply(lambda x: x.median())
            elif v == "count":
                df_contents[k] = grps[k].apply(lambda x: x.count())
        return pd.DataFrame(df_contents)

    @staticmethod 
    def transform_to_py(df, col, col_spec):
        commands = [
            "    grps = df.groupby('%s')" % col,
            "    df_contents = {}"
        ]
        for k, v in col_spec.items():
            if v == "sum":
                commands.append("    paddydf_contents['%s'] = grps['%s'].apply(lambda x: x.sum())" % (k, k))
            elif v == "mean":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.mean())" % (k, k))
            elif v == "median":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.median())" % (k, k))
            elif v == "count":
                commands.append("    df_contents['%s'] = grps['%s'].apply(lambda x: x.count())" % (k, k))
        commands.append("    df = pd.DataFrame(df_contents)")
        return "\n".join(commands)


In [ ]:
!echo $CONDA_DEFAULT_ENV
!pwd

In [ ]:
# notice that there is now a Groupby2 Command in the widget

In [ ]:
df.info()